# NLP Data Science Technical Interview
![NLP Data Science](NLP_data_science.jpg)


## Problem
A fictional B2B company sells products to online shops and wants to create a list of potential customers.
They scrapped a large number of major German web sites and labelled some of the data with a flag denoting if the web site is an online shop or not.


### Task 1
Develop a classifier which is able to predict whether a web site is an online shop by looking at the HTML content of its main page.

### Task 2
Using this classifer, create predictions for each of the web sites which are unclassified so far ("dataset 2"). Provide the prediction as a CSV file containing the domain name and a flag that denotes if the respective web site is an online shop.

### Task 3
Explain your approach and its technical details to our team.

### Task 4
Alas, the VP Sales of the company does not trust black box models and thus wants to understand what the model learned and how it comes to its decisions. In order to convince him, illustrate which content a web page needs to contain or how it needs to be structured to be classified as an online shop by your model. The VP Sales is a non-technical person and does not have deep knowledge about data analysis, so keep this part of the presentation as easy to understand as possible.


**NOTE**: We will combine task 3 into task 1, so that as we build the solution, it will also be explained.


## Principles
To solve this problem we will stick to a few basic principles.

* The principle of finding the **lowest hanging fruit**. That is, we will find the the thing that brings the most value with the least effort.
* One the core principles of **Agile**. We will solve the problem **incrementally** and **iteratively**. We don't want to get bogged down in perfecting one aspect and using up all of our time obsessing over one thing.

In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

import pandas as pd

from utils.data_utils import DataUtils
from utils.HTML_extractor import strip_tags


First things first, let's have a look at our data! How about we start with the training data csv then the unclassified data set?

In [2]:
CLASSIFIED_CSV_PATH = 'data/dataset1.csv'

classified_df = DataUtils.load_csv(CLASSIFIED_CSV_PATH)
classified_df


,domain,is_shop
0,111.com,0
1,12xl.de,1
2,1a-buerotechnik.de,1
3,1a-yachtcharter.de,0
4,1blu.de,0
...,...,...
855,zeus-zukunft.de,0
856,zhaw.ch,0
857,zoeliakie-gourmet.at,1
858,zookauf-zwahr.de,1


In [3]:
number_of_entries = len(classified_df)
number_of_shops = len(classified_df[classified_df["is_shop"] == 1])
percent_of_shops = (number_of_shops / number_of_entries)*100
print(
    f'out of {len(classified_df)} entries, {percent_of_shops}% are shops')


out of 860 entries, 50.0% are shops


In [4]:
UNCLASSIFIED_CSV_PATH = 'data/dataset2.csv'

unclassified_df = DataUtils.load_csv(UNCLASSIFIED_CSV_PATH)
unclassified_df

,domain
0,77records.de
1,absperrtechnik24.de
2,ackermedia.de
3,acris-ecommerce.at
4,adepto-shop.de
...,...
195,xeon-hosting.de
196,xodox.de
197,zaubermode.de
198,zipf-immobilien24.de


Okay dokey, we have two csv files we loaded into pandas that contains the domain names and the flags denoting if the respective web site is an online shop. How about we check out an HTML file?

In [5]:
EXAMPLE_HTML_PATH = 'data/scraped_html/1a-buerotechnik.de.html'

html = DataUtils.load_html(EXAMPLE_HTML_PATH)
html

<!DOCTYPE html>
<html dir="ltr" lang="de"><head>
<!--

		Shopsoftware by Gambio GmbH (c) 2005-2017 [www.gambio.de]

		Gambio GmbH offers you highly scalable E-Commerce-Solutions and Services.
		The Shopsoftware is redistributable under the GNU General Public License (Version 2) [http://www.gnu.org/licenses/gpl-2.0.html].
		based on: E-Commerce Engine Copyright (c) 2006 xt:Commerce, created by Mario Zanier & Guido Winger and licensed under GNU/GPL.
		Information and contribution at http://www.xt-commerce.com

		Please visit our website: www.gambio.de

		-->
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="index,follow,noodp,noydir" name="robots"/>
<meta content="de" name="language"/>
<meta content="K.-R. Kugler" name="author"/>
<meta content="2GrzpAAS3QpIFK9CfFOW6H1zd6mIJXw4EhkiwtvAalQ" name="google-site-verification_"/>
<meta content="(c) 1a-Bürotechnik 2001-2016" name="copyright"/>
<meta content=

Yep, that looks like HTML alright. But it isn't so useful for our purpose of classifying web sites as online shops. We will need to somehow turn this HTML into something useful for our classifier.

**NOTE**: It is interesting to see in the comments of this specific example it mentions `Shopsoftware by Gambio GmbH`, it might be interesting to see if many of them contain such a blatent reference to being a shop. However, we will stick for right now to the task of stripping down the HTML and classifying it as an online shop via the text of the website itself.

**Did you know?** One of the most time consuming steps in NLP data science is the cleaning of text data scraped from websites? 😉

In [6]:
extracted_text = strip_tags(html)
extracted_text

" Lastschrift, Kreditkarte und Rechnungskauf ohne Paypal-Konto - 1a-Bürotechnik Ihr Discount-Versand, schnell-kompetent-preiswert JavaScript ist in Ihrem Browser deaktiviert. Aktivieren Sie JavaScript, um alle Funktionen des Shops nutzen und alle Inhalte sehen zu können. Kundenlogin Kundengruppe: Gast Merkzettel Warenkorb Suchen Über uns ►►► Rechnungskauf für Behörden Go Startseite Ihr Warenkorb keine Produkte LOTUS™ In Kürze bei uns verfügbar! Aktenvernichter HSM Securio B24 1,9x15mm Cross-Cut, Stufe P5 01.01.2018 Aktenvernichter HSM Securio B32 0,78x11mm, Stufe P6, autom. Ölung 01.01.2018 Unsere Empfehlungen AVerVision F17-8M Full HD, AVer... 349,98 EUR LED-Deckenleuchte MAULstart, weiß, 120 cm, 35 W 82,80 EUR Aktenvernichter HSM Securio AF500, 1,9x15mm,... 749,98 EUR Mobiles Whiteboard MAULstandard Emaille,... 318,73 EUR Aktenvernichter Fellowes Powershred M-7C... 49,73 EUR Tonrec Toner 31010L ersetzt Lexmark T640,... 64,28 EUR OLYMPIA Registrierkasse Touch 200 weiß,... 859,00 EUR A

Looking at a few other examples, it looks pretty good for our purpose. Data can always be cleaner,but let's not get stuck cleaning this data further at this time and say it is good enough for now. We will see if we need to do some additional cleaning further down the road. 

Now that we have a way to read and clean up the HTML into normal text strings, we should probably apply that to all of the HTML data and while we are at it, merge it to the classified and unclassified csvs.

To do this we will:
* Get a file list from the HTML files in the directory
* Strip the ".html" extension from the file names
* Match those stripped file names with the domain names in the csvs
* Merge the parsed HTML to the csvs into a new column called `text`

Getting the file list is pretty straight forward. We will write a function that does this for us and throw it into the the `data_utils.py` file. Here's the output (for the first 10):

In [7]:
SCRAPED_DATA_DIR = 'data/scraped_html/'

file_list = DataUtils.get_file_list(SCRAPED_DATA_DIR)
print(file_list[:10])

['toskanaferien.de.html', 'pc-ostsee.de.html', 'hammerkauf.de.html', 'mobile-laden.com.html', 'lutter.net.html', 'meinemarkenmode.de.html', 'sonnenhotels.de.html', 'gelesi.de.html', 'perfectmallorca.de.html', 'bellanea.de.html']


We will need to write a function that will go through the list of files in the directory and returns a dataframe with the domains and text.

**NOTE**: In the original files, there was a `.ipynb_checkpoints` directory in the `data/scraped_html` directory that we will not be using, so I tossed it out.

In [8]:
text_df = DataUtils.process_html_files(SCRAPED_DATA_DIR)
text_df


,domain,text
0,toskanaferien.de,Home - Toskanaferien HomeUrlaubsort Lari Dolc...
1,pc-ostsee.de,PCO - Privat Charter Ostsee - Yachtcharter Di...
2,hammerkauf.de,Hammerkauf Online-Shop - Baumarkt 02251 650 1...
3,mobile-laden.com,Mobile-Laden.com - Handyzubehör zu Top Preise...
4,lutter.net,Immobilien Makler Rostock Häuser Wohnungen Ka...
...,...,...
1055,nakur.de,Nakur Warenkorb 0 Artikel Ihr Konto Kasse Anm...
1056,babyviduals.de,"Babynahrung tiefgekühlt von Babyviduals, Bio-..."
1057,taschenkaufhaus.de,"Taschen, Rucksäcke und Reisegepäck: Taschenka..."
1058,gastroimmobilien.net,gastroimmobilien.net - Gastro Immobilien Das ...


This dataframe looks pretty good. Next we will merge this dataframe with the training and unclassified dataframes by the `domain` column.

In [9]:
classified_text_df = pd.merge(classified_df, text_df, on="domain")
unclassified_text_df = pd.merge(unclassified_df, text_df, on="domain")

In [10]:
classified_text_df

,domain,is_shop,text
0,111.com,0,dns-net.ch | DNS-NET Services GmbH Our domain...
1,12xl.de,1,Bekleidung in Übergrößen für Herren | Herrenm...
2,1a-buerotechnik.de,1,"Lastschrift, Kreditkarte und Rechnungskauf oh..."
3,1a-yachtcharter.de,0,Yachtcharter - 9196 Yachten online chartern M...
4,1blu.de,0,Neue Internetpräsenz. hosted by www.1blu.de Hi...
...,...,...,...
855,zeus-zukunft.de,0,Zeus Zukunft - Ihr Spezialist für Einkommenss...
856,zhaw.ch,0,Willkommen an der ZHAW | ZHAW Zürcher Hochsch...
857,zoeliakie-gourmet.at,1,Vital Gourmet 03177 25 295 / 0676 60 45 9 45 ...
858,zookauf-zwahr.de,1,Startseite 03581 / 405152 Login Kontakt Waren...


In [ ]:
classified_text_df.to_csv('data/classified_text_df.csv')

In [11]:
unclassified_text_df

,domain,text
0,77records.de,DJ Equipment | DJ Zubehör ★ 77records.de Um 7...
1,absperrtechnik24.de,"Absperrpfosten, Schilder, Fahrradständer und ..."
2,ackermedia.de,Hosting - Avernis Domain bestellen / Domains ...
3,acris-ecommerce.at,E-Commerce & Shopware Agentur aus Linz OÖ ► A...
4,adepto-shop.de,ADEPTO - Reinigungsfachhandel - Versandkosten...
...,...,...
195,xeon-hosting.de,Xeon-Hosting.de TeamSpeak3 Live Chat Warenkor...
196,xodox.de,"XODOX - Webdesign, Server, E-Mail & SEO aus Fo..."
197,zaubermode.de,kunterbunte Kindermode und Babykleidung - gün...
198,zipf-immobilien24.de,Zipf und Partner Immobilien GmbH - Immobilien...


In [ ]:
unclassified_text_df.to_csv('data/unclassified_text_df.csv')

Uh huh, looking pretty good with these dataframes. I'd say we are ready for the next major step...

# Task 1
(with Task 3)

Let's do some machine learning stuff! Well, almost.. There is the matter of figuring out our approach and of course doing the pre-processing that needs to be done.

Sticking with our principles, we will start with the easiest solution and then we will benchmark it and see where it gets us. So instead of doing some crazy SOTA stuff, we will encode the the text into TF-IDF vectors and use that as features for a "classic" intent classifier. 

This is very similar to the task of classifying emails as `spam` or `ham` for another binary classification example, but it can also be considered similar to doing multi-class intent matching that could be used in NLU for tasks found in matching utterances to intents like in a voice assistant. 

### Why are we using TF-IDF?
Well based on experience, we have found that TF-IDF is pretty good at being the features for an intent classifier in tasks like this, yet is super easy to do and computationally inexpensive. It is usually a good place to start.

Our **hypothesis** here is that ranking the terms in the documents by their frequency while lowering the rank of terms that are found in a lot of documents, while disregarding the word order is a good way to get a sense of the overall importance of the terms and therefore makes good features. So this is better than a simple bag of words (BoWs) approach, which would just be getting the frequeny of each term. However it might not be as good as other approaches that might use deeper context to get more features (such as word embeddings). But then again, it could give us really great results!

Of course we could also use stop words with BoWs or even with TF-IDF, but we will stick to TF-IDF for now and see where that gets us. We just love those low hanging fruits!

### Application
To do all of this, we will go with the easy to use `sklearn` library. It is what we like to call "old school cool". It is a great way to get started and explore the relationships of the features, train models, and evaluate them. Luckily, I happen to have written an NLU engine that is open source and uses the same library, so we will use parts of that. Please see this [Secret Sauce AI repo](https://github.com/secretsauceai/NLU-engine-prototype-benchmarks) for more information (watch out: this NLU project is still a major work in progress!). Because the project isn't done yet, there is no pipy package for it yet, so we will just grab the pieces we want and go from there.


Let's load the training and unclassified dataframes from the previous step.

In [16]:
classified_text_df = DataUtils.load_csv('data/classified_text_df.csv')
unclassified_text_df = DataUtils.load_csv('data/unclassified_text_df.csv')

## Preprocessing TF-IDF

Kicking this step off, we will first create a TF-IDF vectorizer.

In [17]:
from nlu_engine import TfidfEncoder

In [18]:
tfidf_vectors, tfidf_vectorizer = TfidfEncoder.create_vectorizer(
    classified_text_df['text'])

To have a better understanding, let's have a look at the top 100 TF-IDF terms in the training data.

In [19]:
top_100_features = TfidfEncoder.get_top_n_features(
    tfidf_vectorizer, tfidf_vectors, 100)
top_100_features

/home/bartmoss/code/NLP_technical_interview/.venv/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


array(['dc', 'tattoos', '2018', 'auswandern', 'amp', '18', 'hotels',
       'shopper', 'nächte', 'bidok', 'akku', 'leverkusen', 'teppiche',
       'euronics', 'arrangement', 'owatrol', 'kettler', 'weber', 'fl',
       'pickerd', 'name', 'alanya', 'ansehen', 'tee', 'teleclinic',
       'nestlé', 'caparol', 'wandtattoo', 'personen', 'bio', 'teneriffa',
       'thermomix', 'uvstars', 'horizon', 'teneriffa', 'spreewald',
       'bali', 'backyard', '000', 'tui', 'samova', 'emsland', 'hotze',
       'prix', 'doppler', 'stufenmatten', 'laminat', 'lidl', 'hairfor2',
       'gundel', 'shoppen', 'duftholz', 'mli', 'produktgruppen', 'braun',
       'hinzufügen', 'kochwerte', 'conow', 'sae', 'kw', 'sprachreisen',
       'internetpräsenz', 'torsino', 'mille', 'produktgruppen', 'eur',
       'ferienhaus', '2018sa', 'spalte', 'bioratio', 'anleitung',
       'weingarten', 'kundenreservierung', 'heizstrahler', 'pharma',
       'zhaw', 'ledlenser', 'azoren', 'cloud', 'unavailable', 'adessa',
       'gas

In [ ]:
#TODO: refactor the NLU Engine stuff to fit with this project (ie remove and rename stuff)

## Machine learning
Ohhhh yeah, we are ready to go! Well almost... Let's have a quick at a boring math lesson first.  

One important thing in the classification of machine learning classifiers is the difference between `descrimintive` and `generative` classifiers. In my opinion, this is one of the most helpful ways to categorize the mathematics behind probabilistic machine learning classifiers.

`Descriminative`: The classifier uses the conditional probability of the features to predict the class directory via the posterior probability of the class, `p(y|x)`. Simply put, it uses the direct inputs, `x` to determine the class.

`Generative`: The classifier uses the joint probability of the inputs, `x` and labels `y` to calculate `p(x,y)` using Bayes rule to pick the most likely label. As the name implies, it is trying to generate the inputs, `x` from the labels, `y`.

Got it? Alright let's go!

### The first classifier example
We will load the modified NLU engine and train a model. We will use `Logistic Regression` for this first example. Why? Because it is a good baseline model that is easy to train and easy to evaluate.

It is very fast to train and run inference on, the performance is pretty outstanding given how light the model is . This is actually why several NLU engines use this for probabilistic inference of intent, such as [Snips NLU](https://github.com/snipsco/snips-nlu).

`Logistic Regression` itself is a discrimitive model. It is a supervised learning model that is used to predict the class of a given sample based on the features of that sample.

Much as the name implies, it uses a logit (logarithmic) function to map linear combinations of features(ie the TFIDF terms here) to probabilities of belonging to a particular class. In our case it is a binary classification problem, meaning that there are only two classes.

In short, we ❤️ `Logistic Regression`!


In [ ]:
#TODO: re-write code for intent classification
#TODO: add in way to evaluate all models and get a report on the results
#TODO: select the best one, train it on all of the classified data, then run it on the unevaluated data and look at a random sampling of the results and report on the results (use ipysheet with boolean column for evaluation)

In [13]:
from nlu_engine import NLUEngine
from nlu_engine import IntentMatcher
from nlu_engine import LR
from nlu_engine import Analytics

In [20]:
LR_model = IntentMatcher.train_classifier(LR, tfidf_vectors, classified_text_df['is_shop'])

Training LogisticRegression(random_state=0, solver='liblinear')


We can breifly check out the prediction of labels here.

In [21]:
IntentMatcher.predict_labels(LR_model, tfidf_vectors)

Predicting labels


array([0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0,
       1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1,
       0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0,
       1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0,
       0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1,
       0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1,
       1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0,
       0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1,
       1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1,
       0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1,
       1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1,

### Analytics

What good is machine learning if we can't benchmark and compare our results? Well, we can!

Let's first demonstrate how to do this with `Logistic Regression` and go step-by-step, then we will put it all together.

In [22]:
predictions = Analytics.cross_validate_classifier(LR, tfidf_vectors, classified_text_df['is_shop'])

Cross validating with LogisticRegression(random_state=0, solver='liblinear')
Time it took to cross validate LogisticRegression(random_state=0, solver='liblinear'): 0.7349064350128174


In [23]:
report = Analytics.generate_report(
    LR, predictions, classified_text_df['is_shop'])
report

Generating report for LogisticRegression(random_state=0, solver='liblinear')


{'0': {'precision': 0.9381898454746137,
  'recall': 0.9883720930232558,
  'f1-score': 0.9626274065685163,
  'support': 430},
 '1': {'precision': 0.9877149877149877,
  'recall': 0.9348837209302325,
  'f1-score': 0.9605734767025089,
  'support': 430},
 'accuracy': 0.9616279069767442,
 'macro avg': {'precision': 0.9629524165948007,
  'recall': 0.9616279069767442,
  'f1-score': 0.9616004416355126,
  'support': 860},
 'weighted avg': {'precision': 0.9629524165948008,
  'recall': 0.9616279069767442,
  'f1-score': 0.9616004416355125,
  'support': 860}}

In [36]:
ENCODING = 'TFIDF'
report_df = Analytics.convert_report_to_df(classifier=LR, report=report, encoding=ENCODING)
report_df

,is_shop,precision,recall,f1-score,support,classifier,encoding
0,0,0.938190,0.988372,0.962627,430.000000,LogisticRegression,TFIDF
1,1,0.987715,0.934884,0.960573,430.000000,LogisticRegression,TFIDF
2,accuracy,0.961628,0.961628,0.961628,0.961628,LogisticRegression,TFIDF
3,macro avg,0.962952,0.961628,0.961600,860.000000,LogisticRegression,TFIDF
4,weighted avg,0.962952,0.961628,0.961600,860.000000,LogisticRegression,TFIDF


We have crammed quite a bit of information into this dataframe, so let's take a look at it and discuss it a bit before we go any further.

The formating of the dataframe isn't perfect, but I personally prefer to keep all the information in one dataframe over having to look at multiple dataframes, and in my opinion it looks nicer than outputting a dictionary like we did above.

We have here our two classes for `is_shop`, that is `0` and `1`. Notice the `accuracy`, `macro avg`, and `weighted avg` are in the same column as `is_shop`. This isn't correct, however as stated, it is just nice to have all of this in one dataframe, so I think we can live with that.

##### What fraction of positive predictions are true predictions?
The `precision` is the proportion of *true positives* over *true positives* and *false positives*.

##### Out of all the true positives, what fraction of them did we identify?
The `recall` is the proportion of *true positives* over *true positives* and *false negatives*.


The `f1-score` is the harmonic mean of precision and recall (2 times the `precision` times the `recall` over the addition of the `precision` and the `recall`). If the `f1-score` is high, it means that the model is doing a good job at predicting the class of a given sample. This is the defacto standard metric for evaluating the performance of a model.

The `support` is the number of samples of the given class that were used to compute the `precision`, `recall`, and `f1-score`. NOTE: the `accuracy` row does not contain the support, but the accuracy across the row.

The `accuracy` is the proportion of samples that were correctly predicted. This is a great measure when you have a binary classification problem and one class isn't more important than another. It isn't so good when you would be dealing with life and death though!

The `macro avg` for the `f1-score` is a metric of the f1 score that calculates independently of the classes, then averages them together.

Whereas `weighted avg` for the `f1-score` is a metric of the f1 score that calculates the f1 score for each class taking into account their `support`, then averages them together. 

Since our classes are balanced, this isn't really interesting, and it can be easily seen that the `macro avg` is the same as the `weighted avg` for our data.


For comparsions we will stick to `macro avg f1-score`.

With this all in mind, we can put it all together to see how well our model performs and get a dataframe of those results.

In [37]:
report_df = NLUEngine.evaluate_intent_classifier(tfidf_vectors=tfidf_vectors, labels=classified_text_df['is_shop'], classifier=LR, encoding=ENCODING)
report_df

Cross validating with LogisticRegression(random_state=0, solver='liblinear')
Time it took to cross validate LogisticRegression(random_state=0, solver='liblinear'): 1.425893783569336
Generating report for LogisticRegression(random_state=0, solver='liblinear')


,is_shop,precision,recall,f1-score,support,classifier,encoding
0,0,0.938190,0.988372,0.962627,430.000000,LogisticRegression,TFIDF
1,1,0.987715,0.934884,0.960573,430.000000,LogisticRegression,TFIDF
2,accuracy,0.961628,0.961628,0.961628,0.961628,LogisticRegression,TFIDF
3,macro avg,0.962952,0.961628,0.961600,860.000000,LogisticRegression,TFIDF
4,weighted avg,0.962952,0.961628,0.961600,860.000000,LogisticRegression,TFIDF


### Benchmark them all
Now we can simply run this over a bunch of different classifiers and see which one performs best.

For this I have selected the following classifiers:
* `Gaussian Naive Bayes` is a generative classifier, I would guess that it won't perform as well as `Logistic Regression` for example, and perhaps will perform the worst. It can be good when you have really sparse data though!
* `Logistic Regression` which has already been discussed.
* `Decision Tree` is descriminative.
* `Random Forest` is another personal favorite of mine. It is a descriminative classifier. As the name implies, it randomly bags up features to form descrion trees via permutation. It is known as an ensemble classifier because it takes the permutated trees and ensembles them together into a forest.
* `K-Nearest Neighbors` is a descriminative classifier. As the name tells us, it uses the nearest neighbors to predict the class of a given sample.
* `AdaBoost` is another tree based discriminative classifier.
* `Support Vector Machine` is a descriminative classifier. It uses hyperplanes to draw boundries between classes based on the support, hench the name. It is a bit more computationally expensive than the other classifiers, but it is a great classifier. I would hypothesize that it could outperform the other classifiers by a small margin.

In [39]:
from nlu_engine import NB, DT, RF, KN, ADA, SVM

classifiers = [LR, DT, RF, KN, ADA, SVM, NB]


In [40]:
classifier_reports_df = NLUEngine.evaluate_all_classifiers(classifiers=classifiers, x_train=tfidf_vectors, y_train=classified_text_df['is_shop'], encoding=ENCODING)

Cross validating with LogisticRegression(random_state=0, solver='liblinear')
Time it took to cross validate LogisticRegression(random_state=0, solver='liblinear'): 1.243239164352417
Generating report for LogisticRegression(random_state=0, solver='liblinear')
Cross validating with DecisionTreeClassifier(random_state=42)
Time it took to cross validate DecisionTreeClassifier(random_state=42): 3.623929262161255
Generating report for DecisionTreeClassifier(random_state=42)
Cross validating with RandomForestClassifier(random_state=0)
Time it took to cross validate RandomForestClassifier(random_state=0): 6.735150575637817
Generating report for RandomForestClassifier(random_state=0)
Cross validating with KNeighborsClassifier(n_neighbors=100)
Time it took to cross validate KNeighborsClassifier(n_neighbors=100): 0.2825195789337158
Generating report for KNeighborsClassifier(n_neighbors=100)
Cross validating with AdaBoostClassifier(n_estimators=100)
Time it took to cross validate AdaBoostClassifie

In [41]:
classifier_reports_df.to_csv('data/classifier_reports_df.csv')

We can skip running this and go straight to the reports.

In [42]:
classifier_reports_df = DataUtils.load_csv('data/classifier_reports_df.csv')

Let's take a look at the results.

In [43]:
classifier_reports_df[classifier_reports_df['is_shop'] == 'macro avg'].sort_values(by='f1-score', ascending=False)

,is_shop,precision,recall,f1-score,support,classifier,encoding
3,macro avg,0.974788,0.974419,0.974414,860.0,RandomForestClassifier,TFIDF
3,macro avg,0.971239,0.970930,0.970925,860.0,AdaBoostClassifier,TFIDF
3,macro avg,0.962952,0.961628,0.961600,860.0,LogisticRegression,TFIDF
3,macro avg,0.960860,0.959302,0.959268,860.0,SVC,TFIDF
3,macro avg,0.954392,0.952326,0.952271,860.0,KNeighborsClassifier,TFIDF
3,macro avg,0.943083,0.943023,0.943021,860.0,DecisionTreeClassifier,TFIDF
3,macro avg,0.913920,0.911628,0.911505,860.0,GaussianNB,TFIDF


Whoa, `Random Forest` for the win! I was actually a bit suprised, as from my personal experience `Support Vector Machine` is often the best classifier for problems like this. It goes to show, you never can tell. However, it was true that the Gaussian Naive Bayes classifier performed the worst.

It would be interesting to fine tune the parameters of `Random Forest` with a grid search and cross validation to see how well it can possibly perform. However this isn't a Kaggle competition and time is limited, so we won't do that here. I think the score is high enough for this example. If time would have allowed, I would have loved to try out using these methods with Word2Vec and I would have especially loved to compare this to more modern classification approaches such as can be found with 🤗 HuggingFace.

# Task 2

And now it's time to use our `Random Forest` classifier to predict the class of the given unlabeled dataset.

## Classification of the unclassified data

In case we skipped ahead, we can load most of the stuff we need for this task here

In [2]:
from nlu_engine import TfidfEncoder
from nlu_engine import NLUEngine
from nlu_engine import IntentMatcher
from nlu_engine import RF

classified_text_df = DataUtils.load_csv('data/classified_text_df.csv')
unclassified_text_df = DataUtils.load_csv('data/unclassified_text_df.csv')

tfidf_vectors, tfidf_vectorizer = TfidfEncoder.create_vectorizer(
    classified_text_df['text'])


Okay dokey, let's train the `Random Forest` classifier. NOTE: we are using all of the labeled data. Since we have really great cross validated benchmarks, we can skip doing any train/test splits and maximize the value of the data we have to make predictions on the unlabeled dataset.

In [3]:
RF_model = IntentMatcher.train_classifier(RF, tfidf_vectors, classified_text_df['is_shop'])

Training RandomForestClassifier(random_state=0)


In [4]:
unclassified_text_vectors = TfidfEncoder.get_vectors(unclassified_text_df['text'], tfidf_vectorizer)

In [31]:
unclassified_text_predicted_df = unclassified_text_df.copy()

In [32]:
unclassified_text_predicted_df['is_shop_predicted'] = IntentMatcher.predict_labels(
    RF_model, unclassified_text_vectors)


Predicting labels


In [34]:
unclassified_text_predicted_df.to_csv('data/unclassified_text_predicted_df.csv')

## Analysis of the predictions

So how'd we do? Let's find out!

We are going to use a widget called `IPySheet` to display the dataframe nicely and create a boolean column (also just called a button, LOL) for ground truthing our predictions. The column will be called `is_shop`.

In [7]:
import ipysheet

In [8]:
review_classifications_sheet = DataUtils.create_sheet(
    unclassified_text_predicted_df)
review_classifications_sheet


Sheet(cells=(Cell(column_end=0, column_start=0, numeric_format=None, row_end=199, row_start=0, squeeze_row=Fal…

In [9]:
reviewed_classifications_df = DataUtils.convert_sheet_to_dataframe(review_classifications_sheet)

In [12]:
reviewed_classifications_df.to_csv('data/reviewed_classifications_df.csv')

After going through and ground truthing our unclassified data, I was unsure of some of the sites. We will remove these from consideration. I would be curious to know how many of the sites were actually shops.

In [13]:
reviewed_classifications_df[reviewed_classifications_df['ambiguous'] == 1]

,domain,is_shop_predicted,preview_text,is_shop,ambiguous
9,allstars-direktvertrieb.de,1,Allstars-Vertrieb Um Allstars-Vertrieb in vol...,0,1
12,baender24.de,1,Baender24.de Diese Seite benötigt Cookies. Si...,0,1
14,bagel.de,0,Bagel Gruppe OkBagel verwendet Cookies. Mit d...,0,1
31,conmoto.com,0,Conmoto - Home page JavaScript scheint in Ihr...,0,1
32,creative-beads.com,1,| Creative Beads Perlen und Schmuckzubehör Ge...,0,1
37,der-fellmann.de,1,Hochwertige Lammfell & Naturfellprodukte bei ...,0,1
39,deutschesportonline.de,0,Startseite | Deutsche Sport Online GmbH Deuts...,0,1
51,fischers-lagerhaus.de,1,Startseite .fb_hidden{position:absolute;top:-...,0,1
54,fuxtec.de,1,FUXTEC Gartengeräte Sehr gut4.69/5.00Zertifiz...,0,1
71,hundeleben-exklusiv.de,1,Exklusive Hundemode | Hundetaschen Luxus | Hu...,0,1


In [16]:
reviewed_classifications_df = reviewed_classifications_df[
    reviewed_classifications_df['ambiguous'] == 0]


In [19]:
incorrectly_classified_df = reviewed_classifications_df[reviewed_classifications_df['is_shop_predicted'] != reviewed_classifications_df['is_shop']]
incorrectly_classified_df

,domain,is_shop_predicted,preview_text,is_shop,ambiguous
15,baulando.com,1,"BAULANDO - Arbeitsschutz, Baumaschinen, Diama...",0,0
48,estugo.de,1,"Onlineshop Hosting für Gambio, Shopware, JTL ...",0,0
67,hollandrad.de,1,Hollandräder in toller Auswahl! | Hollandrad ...,0,0
88,kreativgewerbe.de,0,"<!--table,div,span,font,p{display:none} -->Pl...",1,0
92,kuechle.de,0,"Küchle - Backen, Dekorieren, Oblaten, Esspapi...",1,0
105,medherbs.de,0,MedHerbs - Bestes aus der Natur Bestes aus der...,1,0
111,modellbau-amstetten.at,0,Startseite Angebot Kontakt Anfahrt Impressum ...,1,0
160,springerle.com,0,www.springerle.com - Herzlich willkommen! Wil...,1,0


In [27]:
number_of_entries = len(reviewed_classifications_df)
percent_correct = round(((number_of_entries - len(
    incorrectly_classified_df)) / number_of_entries), 2)*100

print(
    f'From a total of {len(reviewed_classifications_df)} entries, {percent_correct}% were correctly classified')


From a total of 186 entries, 96.0% were correctly classified


I don't want to dive any deeper into analysis of this data, as I don't feel like my ground truthing might be 100% as there were so many ambiguous sites for me. But I would say this percentage of correct classifications would fall within our expected range.